# .NET Interactive AI Getting started Guide

This guide will help configure your Azure OpenAI resources for use with the .NET Interactive AI extensions. 

## Prerequisites

- [Azure Account](https://aka.ms/free)
- [Azure OpenAI Service](https://learn.microsoft.com/azure/cognitive-services/openai/overview#how-do-i-get-access-to-azure-openai)
    - [Deployment](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource)
        - GPT
        - ChatGPT
        - Embeddings
    - DALL-E

For more details on getting your endpoint and key values, see the [Azure OpenAI documentation](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?tabs=command-line&pivots=programming-language-csharp#retrieve-key-and-endpoint).

## A brief guide
There are different types of kernels
 - SkillKernel
 - TextCompletionKernel
 - KnowledgeKernel
 - ImageGenerationKernel
 - ChatCompletion
 - VectorStore

### Skill kernel
It is used by the `TextCompletionKernel`, this kernel is used to define prompt templates that can be composed and used by the `TextCompletionKernel`
### TextCompletionKernel
The `TextCompletionKernel` accepts prompts and uses a LLM to generate the answer. The kernel can take use the `Skill kernel`'s prompt templates to calculate the answers.
If is conencted with a `KnowledgeKernel` then it is also automatically fetching relevant facts and uses them in the answer (automatic RAG).
### ImageGenerationKernel
Uses DALL-E 2 to generate images from the prompt
### ChatCompletion
The `ChatCompletion` keeps track of the conversation history. If connected with a `KnowledgeKernel` each conversation turn is marked with the text embeddings for both teh user message and the answer. These text embeddings can be used to:
 - track topic count
 - filter out previous turns that have low relevance with current question (increases focus and reduces token usage)
### KnowledgeKernel
It uses a `text embedding generator` model to calcuate text embeddings and then store the knowledge using and instance of `VectorStore` Kernel (which is require). When creating a new `KnowledgeKernel` it is possible to reuse a instance of `VectorStore`, this is useful when we want to reuse previous data and not recompute the all knowledge.
This kernel is able to store data from verbatim input in a cell, from disk and from url. It also allows to process incoming knowledge and derive more facts. During knowledge processing you can calso use the other AI kernels to generate additional facts.

 
```mermaid
---
title: Kernel relationships
---
classDiagram
    KnowledgeKernel <.. ChatCompletion 
    KnowledgeKernel <.. TextCompletionKernel
    TextCompletionKernel --> SkillKernel
    KnowledgeKernel --> VectorStore
    class SkillKernel
    class TextCompletionKernel
    class KnowledgeKernel
    class VectorStore
    class ChatCompletion
    class ImageGenerationKernel
```

## Connecting kernels

We will now use the api key, endpoint, and deployment names to create kernels

## Connecting kernels

We will now use the api key, endpoint, and deployment names to create kernels

Get the following valuesfrom teh Azure portal:

- Your Azure OpenAI endpoint.
- Your Azure OpenAI key.
- Your Azure OpenAI GPT text completion model name (i.e. text-davinci-003).
- Your Azure OpenAI ChatGPT model name (i.e. gpt35-turbo).
- Your Azure OpenAI Embedding model name (i.e. text-embedding-ada-002).

In [ ]:
#!value --name key
YOUR AZURE OPEN AI KEY

In [ ]:
#!value --name endpoint
https://your-enpoint.openai.azure.com/

### Install .NET Interactive AI extension NuGet package

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23567.4"


### Connect to you deployemnts

connect the TextEmbedding model (for example `text-embedding-ada-002`) to create a `Knowledge` kernel. it will be called `knowledge`. by default this command will also create a `VectorStore` kernel using the `DuckDBVectorStore` implementation and a `DuckDB` kernel too. 

The `VectorStore` kernel will allow to query vectors, export and import knowledge, while the `DuckDB` kernel is providing SQL access to the in memory DB

In [ ]:
#!connect azure-openai --model-type TextEmbeddingGenerator --kernel-name knowledge --api-key @value:key --endpoint @value:endpoint --deployment text-embedding-ada-002

if you have a chatGPT model available you don't need to deploy a `TextCompletion`  model. Connecting a `ChatCompletion` kernel will also create a `TextCompletion` kernel and a `Skill` kernel. 
At this moment the `ChatCompletion` is implementing a chat loop, there is no implicit `RAG` behaviour and the prompts templates won't be used.



In [ ]:
#!connect azure-openai --model-type ChatCompletion --kernel-name chat --api-key @value:key --endpoint @value:endpoint --deployment gpt-35-turbo-16k --use-knowledge knowledge

If you have access to the  `DALL-E` model in Azure Open Ai then you can connect it 

In [ ]:
#!connect azure-openai --model-type ImageGenerator --kernel-name image --api-key @value:key --endpoint @value:endpoint --deployment DALL-E

### Generate responses using models

Select a `TextCompletion` or `ChatCompletion` kernel and send your prompt.

In [ ]:
Who develops .NET Interactive?


### Kernel Apis
#### Skill Kernel
This kernel enabled the definition of prompt templates submitting code like the following snippet
```
#!function stylist --skill writer
Rewrite the following text in the style of {{$person}}:
[BEGIN TEXT]
{{$input}}
[END TEXT]
Please write it in verse.
```
The argument `stylist` is the name of this template; templates can be grouped by skill using the `--skill` option. To define template parameters use the syntax `{{$PARAMETER_NAME}}`. Once the prompt template is evaluate it is ready for use, Look at the `POLYGLOT NOTEBOOK:VARIABLES` (use can use the button `Variables` to open it) and you will see the templates and their list of parameters.
To use the prompts you will need to submit some text to a `TextCompletion` Kernel.



In [ ]:
#!function stylist --skill writer
Answer the question in the style of {{$person}}:
[BEGIN TEXT]
{{$input}}
[END TEXT]
Please write it in verse.

#### TextCompletion Kernel
This kernel will use the LLM to evaluate prompts. To use it just evaluate the following

In [ ]:
Who develops .NET Interactive?

let's do the same but using the prompt template `stylist`. Note that using the `#!set` magic command we can set the template parameter `person`, in this case an input is required to continus.

In [ ]:
#!set --name person --value @input:"Whose style would you like to summarize this in?"
#!use-skills function.writer.stylist
.NET Interactive engine can be used many ways.
* It takes messages, routes them to subkernels, and can format returned data.
* Messages are initiated via a host (like Polyglot Notebooks).
* Subkernels can be a bunch of different languages (including Mermaid).
* Subkernels can share data.
* Subkernels can be running in different processes and on different machines.
* We'd love your ideas on how we can leverage this for developers.